# Manage GlassFlow pipelines

In this notebook, we demonstrate how to manage pipelines using the SDK.

We begin by creating a client with our Personal Access Token

In [ ]:
from glassflow import GlassFlowClient

# Please edit this variable with your own personal access token from https://app.glassflow.dev/profile
personal_access_token = "< your-personal-access-token >"

In [ ]:
client = GlassFlowClient(
    personal_access_token=personal_access_token
)

## Create pipeline

If you have already a pipeline created you can skip this section and jump to [Update pipeline](#update-pipeline) section.

In [ ]:
%%bash
echo "Writing transformation function ..."
cat <<EOT > transformation.py 
def handler(data, log):
    log.info(data)
    return data
EOT
echo "Done!"

In [ ]:
space = client.create_space(name="tutorials")
pipeline = client.create_pipeline(
    name="example pipeline",
    space_id=space.id,
    transformation_file="transformation.py",
)

## Update pipeline



In [ ]:
PIPELINE_ID = pipeline.id
pipeline_to_update = client.get_pipeline(pipeline_id=PIPELINE_ID)

Let's update the transformation code and add an environment variable to the pipeline.

In [ ]:
%%bash
echo "Writing transformation function ..."
cat <<EOT > transformation_with_env.py 
import os


def handler(data, log):
    env = os.getenv("MY_VAR")
    log.info("My env variable: %s", env)
    return data
EOT
echo "Done!"

In [ ]:
updated_pipeline = pipeline_to_update.update(
    transformation_file="transformation_with_env.py",
    env_vars=[
        {
            "name": "MY_VAR",
            "value": "my env var value"
        }
    ]
)

In [ ]:
updated_pipeline.env_vars

In [ ]:
from IPython.display import Markdown as md

md(f"```python\n{updated_pipeline.transformation_code}\n```")

### Pause a pipeline

If you want to pause the pipeline so it does not process any new event, we can do that as follows:

In [ ]:
updated_pipeline = pipeline_to_update.update(
    state="paused"
)
updated_pipeline.state

## Clean-up

Delete the pipelines and spaces we created:

In [ ]:
pipeline.delete()
space.delete()